In [3]:
#Import packages
import pandas as pd
import numpy as np
import glob
import re
import string
import codecs

In [4]:
#Create single parliament_qs dataframe with all question data
li = []

for file in glob.glob('./Parliament_Qs/rajyasabha_questions_and_answers_*.csv'):
    data = pd.read_csv(file)
    li.append(data)
    
parliament_qs = pd.concat(li, axis = 0, ignore_index = True)

C:\Users\deron\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [5]:
#Code borrowed and adapted from George Chen, Carnegie Mellon University#
#Define function to remove punctuation and whitespace, and lowercase all text
def makeWordList(str_object):
    
    corpus_text = str(str_object)
    
    for c in string.punctuation:
        corpus_text = corpus_text.replace(c, "")  # -- (1)
    
    text = re.sub(r'\S*\d\S*','',corpus_text) # -- (2)
    text = re.sub(r'[^\w\s]','',text)         # -- (3)
    
    text = text.lower().split()           # -- (4)         
    
    li = []
    for token in text:
        li.append(token)

    return " ".join(li)

In [6]:
#Process the questions
processed_questions = []

for str_object in list(parliament_qs["question_description"]):
    processed_questions.append(makeWordList(str_object))

In [7]:
#Process responses
processed_answers = []

for str_object in list(parliament_qs["answer"]):
    processed_answers.append(makeWordList(str_object))

In [8]:
#Use CountVectorizer to transform parliamentary questions
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=200, stop_words="english", max_df=0.8)
questions_fit = vectorizer.fit(processed_questions)
X_questions = vectorizer.fit_transform(processed_questions).toarray()

In [9]:
#!Time-consuming!#
#Create topics using LDA
num_topics = 10

from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=num_topics, learning_method='online', random_state=0)
lda.fit(X_questions)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=10, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [11]:
#Display top 10 words from each topic
words = list(questions_fit.vocabulary_)
topic_word_distributions = np.array([row / row.sum() for row in lda.components_])
num_top_words = 10

print('Displaying the top 10 words per topic and their probabilities within the topic...')
print()

for topic_idx in range(num_topics):
    print('[Topic ', topic_idx, ']', sep='')
    sort_indices = np.argsort(-topic_word_distributions[topic_idx])
    for rank in range(num_top_words):
        word_idx = sort_indices[rank]
        print(words[word_idx], ':', topic_word_distributions[topic_idx, word_idx])
    print()

Displaying the top 10 words per topic and their probabilities within the topic...

[Topic 0]
gap : 0.06760957134748051
farmers : 0.0331074729264825
partnership : 0.0325008695861109
infrastructure : 0.03201284098563931
regardc : 0.026739571880887214
energy : 0.02614834419974233
technology : 0.025976113185775272
proportion : 0.020280261709398097
hilly : 0.018522831226632203
december : 0.01675405143255343

[Topic 1]
laid : 0.06525003276359193
regular : 0.05778434814919767
framed : 0.042211316866419255
infrastructure : 0.02711874207911778
yearb : 0.024732822055675906
gap : 0.020124111748620874
schools : 0.019201463114626036
entered : 0.01569709716485583
northeastern : 0.014069795495692275
sites : 0.012737583586373554

[Topic 2]
infrastructure : 0.04616558999782208
proportion : 0.031170419072842886
directed : 0.02838172194357632
hilly : 0.020298604045704306
key : 0.018402013529961927
crore : 0.018241918531560708
based : 0.017801253862481336
gap : 0.017378913579152576
tourist : 0.01183176273

In [ ]:
len(processed_questions)

In [ ]:
X_questions